In [2]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.1/12.8 MB 518.5 kB/s eta 0:00:25
     --------------------------------------- 0.1/12.8 MB 901.1 kB/s eta 0:00:15
     --------------------------------------- 0.2/12.8 MB 833.5 kB/s eta 0:00:16
     --------------------------------------- 0.2/12.8 MB 752.9 kB/s eta 0:00:17
     --------------------------------------- 0.2/12.8 MB 752.9 kB/s eta 0:00:17
     --------------------------------------- 0.2/12.8 MB 752.9 kB/s eta 0:00:17
     --------------------------------------- 0.2/12.8 MB 752.9 kB/s eta 0:00:17
     --------------------------------------- 0.2/12.8 MB 752.9 kB/s eta 0:00:17
      -------------------------------------- 0.2/12.8 MB 428.5 kB/s eta 0:00:30
      -------------------------------------- 0.2/12.8 MB 450.6 kB/s eta 0:00:28
      -------------------------------------- 0.3/12.8 MB

In [4]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string
from heapq import nlargest

In [5]:
nlp = spacy.load("en_core_web_sm")

In [6]:
def calculate_word_frequencies(doc):

    word_frequencies = {}
    stopwords = list(STOP_WORDS)
    punctuation = string.punctuation + '\n'
    
    for word in doc:
        word_text = word.text.lower()
        if word_text not in stopwords and word_text not in punctuation:
            if word_text not in word_frequencies:
                word_frequencies[word_text] = 1
            else:
                word_frequencies[word_text] += 1
    
    return word_frequencies

In [7]:
def normalize_word_frequencies(word_frequencies):
    
    max_frequency = max(word_frequencies.values())
    
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word] / max_frequency
    
    return word_frequencies

In [8]:
def score_sentences(doc, word_frequencies):
    
    sentence_scores = {}
    
    for sent in doc.sents:
        for word in sent:
            word_text = word.text.lower()
            if word_text in word_frequencies.keys():
                if sent not in sentence_scores:
                    sentence_scores[sent] = word_frequencies[word_text]
                else:
                    sentence_scores[sent] += word_frequencies[word_text]
    
    return sentence_scores

In [9]:
def generate_summary(sentence_scores, doc, summary_ratio=0.3):
    
    sentence_tokens = [sent for sent in doc.sents]
    select_length = int(len(sentence_tokens) * summary_ratio)
    summary_sentences = nlargest(select_length, sentence_scores, key=sentence_scores.get)
    final_summary = ' '.join([sent.text for sent in summary_sentences])
    
    return final_summary

In [10]:
text = input("Please enter the text you want to summarize: ")

doc = nlp(text)

word_frequencies = calculate_word_frequencies(doc)

normalized_word_frequencies = normalize_word_frequencies(word_frequencies)

sentence_scores = score_sentences(doc, normalized_word_frequencies)

summary = generate_summary(sentence_scores, doc)

print("Final Summary:", summary)

Please enter the text you want to summarize: Clean water is essential for life. Access to clean and safe drinking water is a basic human right, but millions of people around the world still lack this necessity. Contaminated water can cause serious health problems, including waterborne diseases such as cholera and dysentery. These diseases can be deadly, particularly for young children and those with weakened immune systems.  Efforts to provide clean water to all communities are critical. Governments, non-profit organizations, and private companies are working together to improve water infrastructure, promote hygiene education, and develop innovative technologies for water purification. These initiatives are making a difference, but there is still much work to be done.  Investing in clean water not only improves health outcomes but also supports economic development. Access to clean water can reduce healthcare costs, increase productivity, and enable children to attend school regularly.